In [1]:




import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
import urllib, json

import json
#
#from IPython.display import Image
import numpy as np
import matplotlib.pyplot as plt

from astropy.io import fits

In [2]:
tables = fits.open("AMCVn_datatable_Ramsay_etal_2018.fits")
datatable= tables[1].data

In [3]:
datatable.columns

ColDefs(
    name = 'Source'; format = '15A'
    name = 'Alt. name'; format = '39A'
    name = 'RA_J2000'; format = '11A'; unit = 'hms'
    name = 'Dec_J2000'; format = '11A'; unit = 'dms'
    name = 'RA_J2015.5'; format = '11A'; unit = 'hms'
    name = 'Dec_2015.5'; format = '11A'; unit = 'dms'
    name = 'GDR2sourceID'; format = 'K'; null = -9223372036854775808
    name = 'Period(d)'; format = 'D'
    name = 'Period'; format = 'E'
    name = 'Period flag'; format = '12A'
    name = 'Outbursts?'; format = 'A'
    name = 'Mag.Range'; format = '16A'
    name = 'Reference'; format = '8A'
    name = 'Parallax'; format = 'E'
    name = 'Parallaxerr'; format = 'E'
    name = 'Distance'; format = 'E'
    name = 'Disterr'; format = 'E'
    name = 'E(B-V)'; format = '5A'
    name = 'A_g'; format = '4A'
    name = 'g_q'; format = 'E'
    name = 'M_g'; format = 'E'
    name = 'M_g_err'; format = 'E'
    name = 'FUV_flux_mJy'; format = 'E'
    name = 'FUV_fluxerr'; format = 'E'
    name = 'NUV_fl

In [4]:
# Let's sort the data by epriod like in the table by Ramsay 

In [5]:
datatable.sort(order='Period(d)')

In [6]:
names = datatable['Source']
periodmin = datatable['Period(d)']*24.*60.
nametwo = datatable['Alt. name']
raj2000 = datatable['RA_J2000']
decj2000 = datatable['DEC_J2000']
coordastropy = SkyCoord(raj2000,decj2000, unit=(u.hourangle, u.deg))
radeg = [ i.ra.deg for i in coordastropy]
decdeg = [ i.dec.deg for i in coordastropy]
urllist = []

In [7]:
for r,d in zip(radeg,decdeg):
    url = "https://mast.stsci.edu/tesscut/api/v0.1/sector?ra={ra}&dec={dec}&y=15&x=10&units=px&sector=All".format(ra=r,dec=d)
    urllist.append(url)
    print(url) 

https://mast.stsci.edu/tesscut/api/v0.1/sector?ra=121.59516666666664&dec=15.458749999999998&y=15&x=10&units=px&sector=All
https://mast.stsci.edu/tesscut/api/v0.1/sector?ra=288.6087083333333&dec=24.94536111111111&y=15&x=10&units=px&sector=All
https://mast.stsci.edu/tesscut/api/v0.1/sector?ra=30.217666666666663&dec=-9.408777777777779&y=15&x=10&units=px&sector=All
https://mast.stsci.edu/tesscut/api/v0.1/sector?ra=207.97691666666663&dec=-6.719138888888889&y=15&x=10&units=px&sector=All
https://mast.stsci.edu/tesscut/api/v0.1/sector?ra=188.7275833333333&dec=37.62891666666667&y=15&x=10&units=px&sector=All
https://mast.stsci.edu/tesscut/api/v0.1/sector?ra=287.0711666666666&dec=39.67677777777777&y=15&x=10&units=px&sector=All
https://mast.stsci.edu/tesscut/api/v0.1/sector?ra=233.971125&dec=-14.220055555555556&y=15&x=10&units=px&sector=All
https://mast.stsci.edu/tesscut/api/v0.1/sector?ra=289.7715833333333&dec=48.251666666666665&y=15&x=10&units=px&sector=All
https://mast.stsci.edu/tesscut/api/v0.

In [35]:
c1 = SkyCoord(ra=10*u.degree, dec=9*u.degree, frame='icrs')
c2 = SkyCoord(ra=10*u.degree, dec=9*u.degree, frame='fk5')
c1.separation(c2) 

<Angle 7.36365319e-06 deg>

In [24]:
responsedic = {}
for name,url,rad,decd,periodm in zip(names,urllist,radeg,decdeg,periodmin):
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    print(name,rad,decd,data,len(data['results']))
    subdic = {}
    subdic['ra'] = rad
    subdic['dec'] = decd
    subdic['results'] = data
    subdic['resultsleng'] = len(data['results'])
    subdic['period(m)'] = periodm
    c = SkyCoord(rad, decd,unit='deg',frame='icrs')
    subdic['coord'] = c.to_string('hmsdms')

    if len(data['results']) > 0:
        sectors = []
        for i in data['results']:
            sectors.append(int(i['sector']))
            subdic['sectors'] = sectors
    else:
        sectors = []
    print(sectors)
   
    responsedic[name] = subdic
   


HM Cnc 121.59516666666664 15.458749999999998 {'results': []} 0
[]
V407 Vul 288.6087083333333 24.94536111111111 {'results': [{'sectorName': 'tess-s0014-1-3', 'sector': '0014', 'camera': '1', 'ccd': '3'}]} 1
[14]
ES Cet 30.217666666666663 -9.408777777777779 {'results': []} 0
[]
SDSS J1351-0643 207.97691666666663 -6.719138888888889 {'results': []} 0
[]
AM CVn 188.7275833333333 37.62891666666667 {'results': []} 0
[]
SDSS J1908+3940 287.0711666666666 39.67677777777777 {'results': [{'sectorName': 'tess-s0014-2-3', 'sector': '0014', 'camera': '2', 'ccd': '3'}]} 1
[14]
HP Lib 233.971125 -14.220055555555556 {'results': []} 0
[]
PTF1 J1919+4815 289.7715833333333 48.251666666666665 {'results': [{'sectorName': 'tess-s0014-2-4', 'sector': '0014', 'camera': '2', 'ccd': '4'}, {'sectorName': 'tess-s0015-2-3', 'sector': '0015', 'camera': '2', 'ccd': '3'}]} 2
[14, 15]
ASASSN-14cc 324.9510416666666 -59.992472222222226 {'results': [{'sectorName': 'tess-s0001-2-1', 'sector': '0001', 'camera': '2', 'ccd': '

In [33]:
with open('ramsaytoprint.txt','w') as file:
    for j,i in enumerate(responsedic):
        resultsleng = responsedic[i]['resultsleng']
        if resultsleng > 0:
            text = '{} {} {} {} {} (min)'.format(j,i,responsedic[i]['coord'],responsedic[i]['sectors'],responsedic[i]['period(m)'])
        else:
            text = '{} {} {} {} {} (min)'.format(j,i,responsedic[i]['coord'],resultsleng,responsedic[i]['period(m)'])
        print(text)
        file.write('{}\n'.format(text))
    

0 HM Cnc 08h06m22.84s +15d27m31.5s 0 5.3568 (min)
1 V407 Vul 19h14m26.09s +24d56m43.3s [14] 9.489936960000001 (min)
2 ES Cet 02h00m52.24s -09d24m31.6s 0 10.33686144 (min)
3 SDSS J1351-0643 13h51m54.46s -06d43m08.9s 0 15.7306032 (min)
4 AM CVn 12h34m54.62s +37d37m44.1s 0 17.145532799999998 (min)
5 SDSS J1908+3940 19h08m17.08s +39d40m36.4s [14] 18.084960000000002 (min)
6 HP Lib 15h35m53.07s -14d13m12.2s 0 18.37872 (min)
7 PTF1 J1919+4815 19h19m05.18s +48d15m06s [14, 15] 22.455935999999998 (min)
8 ASASSN-14cc 21h39m48.25s -59d59m32.9s [1] 22.464 (min)
9 CX361 17h51m07.66s -29d40m37.3s 0 22.896000000000004 (min)
10 CR Boo 13h48m55.22s +07d57m35.8s 0 24.521759999999997 (min)
11 KL Dra 19h24m38.25s +59d41m46.7s [14, 15, 16, 17, 18, 19, 20] 25.0272 (min)
12 PTF1 J2219+3135 22h19m10.09s +31d35m23.1s [16] 26.064 (min)
13 V803 Cen 13h23m44.54s -41d44m29.5s [11] 26.6112 (min)
14 PTF1 J0719+4858 07h19m12.12s +48d58m34.7s [20] 26.7696 (min)
15 ASASSN-15kf 15h38m38.24s -30d35m49.7s 0 27.6768 (min)
1

In [30]:
for j,i in enumerate(responsedic):
    print(responsedic[i])


{'ra': 121.59516666666664, 'dec': 15.458749999999998, 'results': {'results': []}, 'resultsleng': 0, 'period(m)': 5.3568, 'coord': '08h06m22.84s +15d27m31.5s'}
{'ra': 288.6087083333333, 'dec': 24.94536111111111, 'results': {'results': [{'sectorName': 'tess-s0014-1-3', 'sector': '0014', 'camera': '1', 'ccd': '3'}]}, 'resultsleng': 1, 'period(m)': 9.489936960000001, 'coord': '19h14m26.09s +24d56m43.3s', 'sectors': [14]}
{'ra': 30.217666666666663, 'dec': -9.408777777777779, 'results': {'results': []}, 'resultsleng': 0, 'period(m)': 10.33686144, 'coord': '02h00m52.24s -09d24m31.6s'}
{'ra': 207.97691666666663, 'dec': -6.719138888888889, 'results': {'results': []}, 'resultsleng': 0, 'period(m)': 15.7306032, 'coord': '13h51m54.46s -06d43m08.9s'}
{'ra': 188.7275833333333, 'dec': 37.62891666666667, 'results': {'results': []}, 'resultsleng': 0, 'period(m)': 17.145532799999998, 'coord': '12h34m54.62s +37d37m44.1s'}
{'ra': 287.0711666666666, 'dec': 39.67677777777777, 'results': {'results': [{'secto

# Save Python dic to json

In [39]:
jsont = json.dumps(responsedic)
f = open("responseramsay.json","w")
f.write(jsont)
f.close()


AttributeError: 'str' object has no attribute 'dumps'